<a href="https://colab.research.google.com/github/vnlvih/Estudos-PySpark/blob/main/02_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 51.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=33681b89a068a7807fd15985aa3faae73c1fb540f03d84f41298c0bcafb9e0bc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# **Data Preparation**

In [4]:
schema = StructType([
                     StructField("mon", IntegerType()),
                     StructField("dom", IntegerType()),
                     StructField("dow", IntegerType()),
                     StructField("carrier", StringType()),
                     StructField("flight", StringType()),
                     StructField("org", StringType()),
                     StructField("mile", DoubleType()),
                     StructField("depart", DoubleType()),
                     StructField("duration", DoubleType()),
                     StructField("delay", DoubleType()),
])

flights = spark.read.csv("/content/drive/MyDrive/Data Science & Afins/DATACAMP/03. Big Data with PySpark/05. Machine Learning with PySpark/00. DataSets/flights.csv", header=True, schema=schema)
flights.show(5)

+---+---+---+-------+------+---+------+------+--------+-----+
|mon|dom|dow|carrier|flight|org|  mile|depart|duration|delay|
+---+---+---+-------+------+---+------+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153.0|  9.48|   351.0| null|
|  0| 22|  2|     UA|  1107|ORD| 316.0| 16.33|    82.0| 30.0|
|  2| 20|  4|     UA|   226|SFO| 337.0|  6.17|    82.0| -8.0|
|  9| 13|  1|     AA|   419|ORD|1236.0| 10.33|   195.0| -5.0|
|  4|  2|  5|     AA|   325|ORD| 258.0|  8.92|    65.0| null|
+---+---+---+-------+------+---+------+------+--------+-----+
only showing top 5 rows



## **Dropping columns**

In [5]:
#Either drop the columns you don't want...
flights = flights.drop("flight")
flights.show(5)
# ... or select  the columns you want to retain 
"""flights_2 = flights.select('mon',
 'dom',
 'dow',
 'carrier',
 'org',
 'mile',
 'depart',
 'duration',
 'delay')
flights_2.show(5)"""

+---+---+---+-------+---+------+------+--------+-----+
|mon|dom|dow|carrier|org|  mile|depart|duration|delay|
+---+---+---+-------+---+------+------+--------+-----+
| 11| 20|  6|     US|JFK|2153.0|  9.48|   351.0| null|
|  0| 22|  2|     UA|ORD| 316.0| 16.33|    82.0| 30.0|
|  2| 20|  4|     UA|SFO| 337.0|  6.17|    82.0| -8.0|
|  9| 13|  1|     AA|ORD|1236.0| 10.33|   195.0| -5.0|
|  4|  2|  5|     AA|ORD| 258.0|  8.92|    65.0| null|
+---+---+---+-------+---+------+------+--------+-----+
only showing top 5 rows



"flights_2 = flights.select('mon',\n 'dom',\n 'dow',\n 'carrier',\n 'org',\n 'mile',\n 'depart',\n 'duration',\n 'delay')\nflights_2.show(5)"

## Filtering out missing data

In [6]:
#How many missing values?

flights.filter("delay IS NULL").count()

2978

In [7]:
#Drop records with missing values in the delay column
flights = flights.filter("delay IS NOT NULL")

#Drop records with missing values in any column

flights = flights.dropna()

## **Mutating columns** 

- Derive a new km column from the mile column, rounding to zero decimal places. One mile is 1.60934 km.
Remove the mile column.
- Create a label column with a value of 1 indicating the delay was 15 minutes or more and 0 otherwise.

In [8]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights = flights.withColumn('label', (flights.delay >= 15).cast('integer'))

# Check first five records
flights.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|    82.0| 30.0| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|    82.0| -8.0| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|   195.0| -5.0|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|   102.0|  2.0| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|   135.0| 54.0|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



## Categorical columns

In the flights data there are two columns, carrier and org, which hold categorical data. You need to transform those columns into indexed numerical values.

In [9]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol="carrier", outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights)

# Indexer creates a new column with numeric index values
flights = indexer_model.transform(flights)

# Repeat the process for the other categorical feature
flights = StringIndexer(inputCol="org", outputCol='org_idx').fit(flights).transform(flights)

## Assembling columns
The final stage of data preparation is to consolidate all of the predictor columns into a single column.

An updated version of the flights data, which takes into account all of the changes from the previous few exercises, has the following predictor columns:

- mon, dom and dow
- carrier_idx (indexed value from carrier)
- org_idx (indexed value from org)
- km
- depart
- duration

In [10]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
    "mon","dom","dow","carrier_idx", "org_idx","km","depart", "duration"
], outputCol='features')

# Consolidate predictor columns
flights = assembler.transform(flights)

# Check the resulting column
flights.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |30.0 |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |-8.0 |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5.0 |
|[5.0,2.0,1.0,0.0,1.0,885.0,7.98,102.0]   |2.0  |
|[7.0,2.0,6.0,1.0,0.0,1180.0,10.83,135.0] |54.0 |
+-----------------------------------------+-----+
only showing top 5 rows



## Train/test split
To objectively assess a Machine Learning model you need to be able to test it on an independent set of data. You can't use the same data that you used to train the model: of course the model will perform (relatively) well on those data!

You will split the data into two components:

- training data (used to train the model) and
- testing data (used to test the model).


Randomly split the flights data into two sets with 80:20 proportions. For repeatability set a random number seed of 17 for the split.
Check that the training data has roughly 80% of the records from the original data.

In [11]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([.8,.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() /flights_test.count()
print(training_ratio)

3.9253168534618204


## Build a Decision Tree
Now that you've split the flights data into training and testing sets, you can use the training set to fit a Decision Tree model.

The data are available as flights_train and flights_test.

NOTE: It will take a few seconds for the model to train… please be patient!

- Import the class for creating a Decision Tree classifier.
- Create a classifier object and fit it to the training data.
- Make predictions for the testing data and take a look at the predictions.

In [12]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier
# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|1    |0.0       |[0.5327142254115661,0.46728577458843396]|
|0    |1.0       |[0.3537163618449412,0.6462836381550588] |
|0    |1.0       |[0.4965986394557823,0.5034013605442177] |
|1    |1.0       |[0.3537163618449412,0.6462836381550588] |
|1    |1.0       |[0.3537163618449412,0.6462836381550588] |
+-----+----------+----------------------------------------+
only showing top 5 rows



## Evaluate the Decision Tree
You can assess the quality of your model by evaluating how well it performs on the testing data. Because the model was not trained on these data, this represents an objective assessment of the model.

A confusion matrix gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

- True Negatives (TN) — model predicts negative outcome & known outcome is negative
- True Positives (TP) — model predicts positive outcome & known outcome is positive
- False Negatives (FN) — model predicts negative outcome but known outcome is positive
- False Positives (FP) — model predicts positive outcome but known outcome is negative.



*   Create a confusion matrix by counting the combinations of label and prediction. Display the result.
*   Count the number of True Negatives, True Positives, False Negatives and False Positives.
*   Calculate the accuracy.





In [13]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy =  (TN + TP) / (TN + TP + FN + FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1687|
|    0|       0.0| 2767|
|    1|       1.0| 3223|
|    0|       1.0| 1870|
+-----+----------+-----+

0.6274222268775531


## Build a Logistic Regression model
You've already built a Decision Tree model using the flights data. Now you're going to create a Logistic Regression model on the same data.

The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

Although you have a variety of predictors at your disposal, you'll only use the mon, depart and duration columns for the moment. These are numerical features which can immediately be used for a Logistic Regression model. You'll need to do a little more work before you can include categorical features. Stay tuned!

The data have been split into training and testing sets and are available as flights_train and flights_test.


- Import the class for creating a Logistic Regression classifier.
- Create a classifier object and train it on the training data.
- Make predictions for the testing data and create a confusion matrix.

In [14]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1671|
|    0|       0.0| 2522|
|    1|       1.0| 3239|
|    0|       1.0| 2115|
+-----+----------+-----+



## Evaluate the Logistic Regression model
Accuracy is generally not a very reliable metric because it can be biased by the most common target class.

There are two other useful metrics:

- precision and
- recall.
Check the slides for this lesson to get the relevant expressions.

Precision is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

Recall is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

The components of the confusion matrix are available as TN, TP, FN and FP, as well as the object prediction.

- Find the precision and recall.
- Create a multi-class evaluator and evaluate weighted precision.
- Create a binary evaluator and evaluate AUC using the "areaUnderROC" metric.

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: "areaUnderROC"})

precision = 0.63
recall    = 0.66


## Punctuation, numbers and tokens
At the end of the previous chapter you loaded a dataset of SMS messages which had been labeled as either "spam" (label 1) or "ham" (label 0). You're now going to use those data to build a classifier model.

But first you'll need to prepare the SMS messages as follows:

remove punctuation and numbers
tokenize (split into individual words)
remove stop words
apply the hashing trick
convert to TF-IDF representation.
In this exercise you'll remove punctuation and numbers, then tokenize the messages.


- Import the function to replace regular expressions and the feature to tokenize.
- Replace all punctuation characters from the text column with a space. Do the same for all numbers in the text column.
- Split the text column into tokens. Name the output column words.

In [16]:
sms = spark.read.csv("/content/drive/MyDrive/Data Science & Afins/DATACAMP/03. Big Data with PySpark/05. Machine Learning with PySpark/00. DataSets/sms.csv", header=False, sep=';')
sms = sms.withColumnRenamed("_c0","id")
sms = sms.withColumnRenamed("_c1","text")
sms = sms.withColumnRenamed("_c2","label")
sms.show(5)

+---+--------------------+-----+
| id|                text|label|
+---+--------------------+-----+
|  1|Sorry, I'll call ...|    0|
|  2|Dont worry. I gue...|    0|
|  3|Call FREEPHONE 08...|    1|
|  4|Win a 1000 cash p...|    1|
|  5|Go until jurong p...|    0|
+---+--------------------+-----+
only showing top 5 rows



In [17]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



### Stop words and hashing
The next steps will be to remove stop words and then apply the hashing trick, converting the results into a TF-IDF.

A quick reminder about these concepts:

- The hashing trick provides a fast and space-efficient way to map a very large (possibly infinite) set of items (in this case, all words contained in the SMS messages) onto a smaller, finite number of values.
- The TF-IDF matrix reflects how important a word is to each document. It takes into account both the frequency of the word within each document but also the frequency of the word across all of the documents in the collection.

In [18]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

wrangled = wrangled.select('id', 'words', 'label')

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms').transform(wrangled)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024).transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features').fit(wrangled).transform(wrangled)

tf_idf.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,384,577,996],[2.273418200008753,3.6288353225642043,3.5890949939146903,4.104259019279279])|
|[dont, worry, guess, busy]      |(1024,[215,233,276,329],[3.9913186080986836,3.3790235241678332,4.734227298217693,4.58299632849377]) |
|[call, freephone]               |(1024,[133,138],[5.367951058306837,2.273418200008753])                                              |
|[win, cash, prize, prize, worth]|(1024,[31,47,62,389],[3.6632029660684124,4.754846585420428,4.072170704727778,7.064594791043114])    |
+--------------------------------+--------------

### Training a spam classifier
The SMS data have now been prepared for building a classifier. Specifically, this is what you have done:

- removed numbers and punctuation
- split the messages into words (or "tokens")
- removed stop words
- applied the hashing trick and
- converted to a TF-IDF representation.

Next you'll need to split the TF-IDF data into training and testing sets. Then you'll use the training data to fit a Logistic Regression model and finally evaluate the performance of that model on the testing data.

In [19]:
tf_idf = tf_idf.withColumn("label", tf_idf.label.cast("int"))

In [20]:
sms = tf_idf.select('label', 'features')

# Split the data into training and test sets
sms_train, sms_test = sms.randomSplit([0.8, 0.2], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the test data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   39|
|    0|       0.0|  932|
|    1|       1.0|  121|
|    0|       1.0|    4|
+-----+----------+-----+

